In [1]:
import pandas as pd
import numpy as np
import torch
import torch.utils.data
import torch.nn as nn
from torch.autograd import Variable
from sklearn.model_selection import train_test_split
import joblib
from tqdm import tqdm
import matplotlib.pyplot as plt
from copy import deepcopy

In [2]:
from isaac.utils import get_cuda_device_if_available, create_directory
device = get_cuda_device_if_available()
print(device)

data_directory = "xy_vxvy_rtheta/"
create_directory(data_directory)
create_directory("models/xy_vxvy_rtheta/")
create_directory("scalers")

cuda:0


In [3]:
from isaac.dataset import read_dataset, prepare_dataset
from isaac.utils import plot_confusion_matrix
from isaac.constants import FORCE_CLASS_COLS, MASS_CLASS_COLS, XY_VXVY_RTHETA_COLS
from isaac.training import training_loop
from isaac.models import MultiBranchModel, initialise_model
from isaac.evaluation import get_best_model_and_its_accuracy

In [4]:
BATCH_SIZE = 128
EPOCHS = 60
NORMALISE_DATA = True
STEP_SIZE = 3
SEQ_END = 2700

INPUT_DIM = 24    # input dimension
HIDDEN_DIM = 25  # hidden layer dimension
N_LAYERS = 4     # number of hidden layers
OUTPUT_DIM = 3   # output dimension
DROPOUT = 0.5

network_params = (INPUT_DIM, HIDDEN_DIM, OUTPUT_DIM, DROPOUT)

## Read dataset and preprocess it

In [5]:
train_trials = read_dataset("data/r_train_trials.h5")
val_trials = read_dataset("data/r_val_trials.h5")

train_classes = [trial.combined_solution.iloc[0] for trial in train_trials]
val_classes = [trial.combined_solution.iloc[0] for trial in val_trials]

np.random.seed(37)
train_trials, _ = train_test_split(train_trials, train_size=4000, stratify=train_classes)
val_trials, _ = train_test_split(val_trials, train_size=1000, stratify=val_classes)

100%|██████████| 10935/10935 [01:16<00:00, 142.49it/s]


## Train model and plot loss and accuracy

In [7]:
N_MODELS = 25

stats_dfs = []
loaders, scaler = prepare_dataset([train_trials, val_trials], 
                                  class_columns=[list(MASS_CLASS_COLS), list(FORCE_CLASS_COLS)], 
                                  multiclass=True,
                                  batch_size=BATCH_SIZE, normalise_data=NORMALISE_DATA,
                                  device=device, training_columns=XY_VXVY_RTHETA_COLS)


for seed in range(N_MODELS):
    df = pd.DataFrame()

    model, error, optimizer = initialise_model(network_params, lr=0.01, seed=seed, device=device, arch=MultiBranchModel)
    epoch_losses, epoch_accuracies, [best_mass_model, best_force_model] = training_loop(model, optimizer, 
                                                                                        error,
                                                                                        loaders[0], loaders[1], 
                                                                                        EPOCHS, seq_end=SEQ_END,
                                                                                        step_size=STEP_SIZE,
                                                                                        multibranch=True)

    torch.save(best_mass_model.state_dict(), "models/xy_vxvy_rtheta/best_mass_model_seed_%d.pt" % seed)
    torch.save(best_force_model.state_dict(), "models/xy_vxvy_rtheta/best_force_model_seed_%d.pt" % seed)
    
    
    train_accuracies = np.array(epoch_accuracies[0])
    val_accuracies = np.array(epoch_accuracies[1]) 
    
    df["Epoch"] = np.arange(EPOCHS)
    df["Mass Loss"] = epoch_losses[:, 0]
    df["Force Loss"] = epoch_losses[:, 1]
    df["Mass Train Accuracy"] = train_accuracies[:, 0]
    df["Mass Val Accuracy"] = val_accuracies[:, 0]
    df["Force Train Accuracy"] = train_accuracies[:, 1]
    df["Force Val Accuracy"] = val_accuracies[:,1]
    df["seed"] = str(seed)
    stats_dfs.append(df)
        
stats = pd.concat(stats_dfs)
stats.to_hdf(data_directory+"stats.h5", key="stats")

100%|██████████| 1000/1000 [00:02<00:00, 335.50it/s]
Train_loss: ([0.42423393 0.30547512]) Train_acc: ([81.11428571 84.45714286]) Val_acc: ([67.7 69.4]): 100%|██████████| 60/60 [09:18<00:00,  9.51s/it]
Train_loss: ([0.44353925 0.28658367]) Train_acc: ([79.88571429 82.37142857]) Val_acc: ([67.9 72.1]): 100%|██████████| 60/60 [09:27<00:00,  9.34s/it]
Train_loss: ([0.4646638  0.26124691]) Train_acc: ([80.34285714 83.11428571]) Val_acc: ([68.6 69.8]): 100%|██████████| 60/60 [09:23<00:00,  9.54s/it]
Train_loss: ([0.41803144 0.28133629]) Train_acc: ([80.54285714 83.54285714]) Val_acc: ([66.3 73.9]): 100%|██████████| 60/60 [09:30<00:00,  9.55s/it]
Train_loss: ([0.44898194 0.25864986]) Train_acc: ([81.05714286 88.        ]) Val_acc: ([68.2 72.6]): 100%|██████████| 60/60 [09:30<00:00,  9.54s/it]
Train_loss: ([0.49371289 0.28662452]) Train_acc: ([79.6        84.57142857]) Val_acc: ([66.8 68.9]): 100%|██████████| 60/60 [09:31<00:00,  9.51s/it]
Train_loss: ([0.42156316 0.2982894 ]) Train_acc: ([79

## Save model and scaler

In [9]:
joblib.dump(scaler, "scalers/passive_xy_vxvy_rtheta_scaler.sk")

['scalers/passive_xy_vxvy_rtheta_scaler.sk']